## Make inference

### Install modules

[TensorFlow SLIM](https://github.com/google-research/tf-slim)


In [ ]:
!pip install tf_slim
!pip install lvis
!pip install tf-models-official

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.1/630.1 kB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
import os
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git
%cd /content/models/
#!git checkout 58d19c67e1d30d905dd5c6e5092348658fed80af
!apt-get update && apt-get install -y -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
!python object_detection/builders/model_builder_test.py

/content
/content/models
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [975 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,029 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/u

### Imports

In [ ]:
import cv2
import zipfile

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
import tensorflow as tf
import numpy as np

from PIL import Image
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

### Unzip and read model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/Parcial2/Models_saved/v3_640X640/Model_3/fine_tuned_model3.zip'
zip_ref = zipfile.ZipFile(path, "r")
zip_ref.extractall("/content/fine_tuned_model")
zip_ref.close()

### Make prediction

In [ ]:
PATH_TO_MODEL_DIR = "/content/fine_tuned_model/content/drive/MyDrive/Parcial2/Models_saved/v3_640X640/Model_3"
PATH_TO_SAVE_MODEL = PATH_TO_MODEL_DIR + "/saved_model"

detect_fn = tf.saved_model.load(PATH_TO_SAVE_MODEL)

### Read label map

In [ ]:
PATH_TO_LABELS = "/content/drive/MyDrive/Parcial2/TFRecord/v3_640x640/animals_label_map.pbtxt"
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

### Read input to evaluate


In [ ]:
def read_image(path):
  image_path = path

  image_np = np.array(Image.open(image_path))

  input_tensor = tf.convert_to_tensor(image_np)
  input_tensor =  input_tensor[tf.newaxis, ...]

  detections = detect_fn(input_tensor)
  num_detections = int(detections.pop('num_detections'))

  return detections, num_detections, image_np

In [ ]:
def show_detection(detections, num_detections, image_np):
  detections2 = {key: value[0,:num_detections].numpy() for key, value in detections.items()}

  detections2['num_detections'] = num_detections

  detections2['detection_classes'] = detections2['detection_classes'].astype(np.int64)

  # Copy image and draw bounding-boxes
  image_np_with_detections = image_np.copy()
  # print(np.array(image_np_with_detections).shape)

  # Use object detection library to show the classification and boundingboxes
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections2['detection_boxes'],
      detections2['detection_classes'],
      detections2['detection_scores'],
      category_index,
      max_boxes_to_draw=15,
      min_score_thresh=0.40,
      use_normalized_coordinates = True
  )


  # Detection result
  image_np_with_detections = cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB)

  return image_np_with_detections


In [ ]:
folder_path = "/content/drive/MyDrive/Parcial2/Imagenes"
#path = "/content/images/frame_1-4_3516.jpg"

count = 0
for im in os.listdir(folder_path):
  path = folder_path + '/' + im
  print(im)
  (detections, num_detections, image_np) = read_image(path)
  image_np_with_detections = show_detection(detections, num_detections, image_np)
  plt.figure()
  cv2_imshow(image_np_with_detections)

  if count == 50:
    break
  count += 1  

Output hidden; open in https://colab.research.google.com to view.